# ATMOSPHERE_NRLMSISE00

## Overview
The `ATMOSPHERE_NRLMSISE00` function provides access to the NRLMSISE-00 empirical model for calculating temperature and density of gases in the atmosphere, from ground level to 1000 km, as a function of time of year, longitude and latitude, solar activity, and earth’s geomagnetic disturbance. NRLMSISE stands for the US Naval Research Laboratory Mass Spectrometer and Incoherent Scatter Radar Exosphere model, released in 2001.

This model is based on measurements other than gravity; it does not provide a calculation method for g or transport properties. The model is defined by its implementation and is widely used in atmospheric and space science.

## Usage
To use this function in Excel, enter the following formula (optional arguments in square brackets):
```excel
=ATMOSPHERE_NRLMSISE00(Z, [latitude], [longitude], [day], [seconds], [f107], [f107_avg])
```

## Arguments
| Argument | Type | Required | Description | Example |
|:---|:---|:---|:---|:---|
| Z | float | Required | Elevation [m] | 1000 |
| latitude | float | Optional | Latitude, between -90 and 90 [degrees] | 45 |
| longitude | float | Optional | Longitude, between -180 and 180 [degrees] | 45 |
| day | float | Optional | Day of year, 0-366 [day] | 150 |
| seconds | float | Optional | Seconds since start of day [s] | 0 |
| f107 | float | Optional | Daily average 10.7 cm solar flux measurement [10^-22 W/m^2/Hz] | 150 |
| f107_avg | float | Optional | 81-day sfu average [10^-22 W/m^2/Hz] | 150 |

## Returns
| Returns | Type | Description | Example |
|:---|:---|:---|:---|
| rho | float | Mass density [kg/m^3] | 1.1019 |
| T | float | Temperature [K] | 285.54 |
| P | float | Pressure, calculated with ideal gas law [Pa] | 101325 |
| He_density | float | Density of helium atoms [count/m^3] | 0.0 |
| O_density | float | Density of monatomic oxygen [count/m^3] | 0.0 |
| N2_density | float | Density of nitrogen molecules [count/m^3] | 0.0 |
| O2_density | float | Density of oxygen molecules [count/m^3] | 0.0 |
| Ar_density | float | Density of Argon atoms [count/m^3] | 0.0 |
| H_density | float | Density of hydrogen atoms [count/m^3] | 0.0 |
| N_density | float | Density of monatomic nitrogen [count/m^3] | 0.0 |
| O_anomalous_density | float | Density of anomalous oxygen [count/m^3] | 0.0 |
| particle_density | float | Total density of molecules [count/m^3] | 0.0 |
| components | list | List of species making up the atmosphere | ['N2', 'O2', ...] |
| zs | list | Mole fractions of each molecule in the atmosphere | [0.78, 0.21, ...] |

## Examples
### Calculating Atmospheric Properties at 1 km Elevation
| Argument | Value |
|:---|:---|
| Z | 1000 |
| latitude | 45 |
| longitude | 45 |
| day | 150 |

```excel
=ATMOSPHERE_NRLMSISE00(1000, 45, 45, 150)
```
Returns: rho = 1.1019, T = 285.54

### Using Default Parameters
| Argument | Value |
|:---|:---|
| Z | 1000 |

```excel
=ATMOSPHERE_NRLMSISE00(1000)
```
Returns: rho ≈ 1.1, T ≈ 285

## Reference
See the `fluids` package [GitHub repository](https://github.com/CalebBell/fluids) and [documentation](https://fluids.readthedocs.io/index.html) for details.  We are grateful to the `fluids` package authors for their work.

In [ ]:
import micropip
await micropip.install('fluids')
from fluids.atmosphere import ATMOSPHERE_NRLMSISE00

def atmosphere_nrlmsise00(Z, latitude=0.0, longitude=0.0, day=0, seconds=0.0, f107=150.0, f107_avg=150.0):
    """
    Calculate atmospheric properties using the NRLMSISE-00 model.

    Args:
        Z (float): Elevation [m]
        latitude (float, optional): Latitude [-90, 90] [degrees]
        longitude (float, optional): Longitude [-180, 180] [degrees]
        day (float, optional): Day of year [0-366]
        seconds (float, optional): Seconds since start of day [s]
        f107 (float, optional): Daily average 10.7 cm solar flux [10^-22 W/m^2/Hz]
        f107_avg (float, optional): 81-day sfu average [10^-22 W/m^2/Hz]

    Returns:
        list: [rho, T, P, He_density, O_density, N2_density, O2_density, Ar_density, H_density, N_density, O_anomalous_density, particle_density, components, zs]
    """
    try:
        result = ATMOSPHERE_NRLMSISE00(Z, latitude, longitude, day, seconds, f107, f107_avg)
        return [
            result.rho, result.T, result.P, result.He_density, result.O_density,
            result.N2_density, result.O2_density, result.Ar_density, result.H_density,
            result.N_density, result.O_anomalous_density, result.particle_density,
            result.components, result.zs
        ]
    except Exception as e:
        return [str(e)] * 14

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_atmosphere_nrlmsise00_basic():
    result = atmosphere_nrlmsise00(1000, 45, 45)
    assert isinstance(result, list)
    assert len(result) == 14
    assert isinstance(result[0], float)
    assert result[0] > 0
    assert isinstance(result[1], float)
    assert result[1] > 0

def test_atmosphere_nrlmsise00_defaults():
    result = atmosphere_nrlmsise00(1000)
    assert isinstance(result, list)
    assert len(result) == 14
    assert isinstance(result[0], float)
    assert result[0] > 0
    assert isinstance(result[1], float)
    assert result[1] > 0

def test_atmosphere_nrlmsise00_error():
    result = atmosphere_nrlmsise00('bad_input')
    assert isinstance(result, list)
    assert all(isinstance(x, str) for x in result)

ipytest.run()

In [ ]:
import gradio as gr

examples = [
    [1000, 45, 45, 150, 0, 150, 150],
    [1000, 0, 0, 0, 0, 150, 150],
]

demo = gr.Interface(
    fn=atmosphere_nrlmsise00,
    inputs=[
        gr.Number(label='Z', value=1000, precision=2),
        gr.Number(label='latitude', value=45, precision=2),
        gr.Number(label='longitude', value=45, precision=2),
        gr.Number(label='day', value=150, precision=2),
        gr.Number(label='seconds', value=0, precision=2),
        gr.Number(label='f107', value=150, precision=2),
        gr.Number(label='f107_avg', value=150, precision=2),
    ],
    outputs=[
        gr.Number(label='rho'),
        gr.Number(label='T'),
        gr.Number(label='P'),
        gr.Number(label='He_density'),
        gr.Number(label='O_density'),
        gr.Number(label='N2_density'),
        gr.Number(label='O2_density'),
        gr.Number(label='Ar_density'),
        gr.Number(label='H_density'),
        gr.Number(label='N_density'),
        gr.Number(label='O_anomalous_density'),
        gr.Number(label='particle_density'),
        gr.Textbox(label='components'),
        gr.Textbox(label='zs'),
    ],
    description="Calculate atmospheric properties using the NRLMSISE-00 model. Returns density, temperature, pressure, and species densities.",
    flagging_mode='never',
    examples=examples
)
demo.launch()